In [ ]:
# 1. Katie Corcoran

In [2]:
# 2.
import psycopg2
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

conn = psycopg2.connect(
database='corgirace',
user='USER',
password='PASSWORD',
host='HOST',
port= '5432'
)
cursor = conn.cursor()
df = pd.read_sql('''
SELECT
    *
FROM corgi''', conn, index_col='corgid')
print(df)

                         corgname  breed gender  weight  age fromfile
corgid                                                               
1                   Snappy Ginger    Pem      F      21  3.5     None
2                   Golden Graham    Pem      M      30  2.0     None
3                          Kraken    Pem     SF      22  3.5     None
4                     Snarla June    Pem     SF      29  3.5     None
5                 Smiley Shortdog  Cardi     NM      33  4.5     None
6                         Stanley    Pem      M      33  3.5     None
7                Holden Corgfield    Pem      M      25  4.0     None
8           Penny the Shedmonster    Pem      F      20  5.0     None
9       Baroness von Fluffyshanks    Pem     SF      22  4.0     None
10                     Prometheus    Pem      M      27  5.0     None
11                 Foxy Stumptail    Pem     NM      29  5.5     None
12                      Drederick    Pem      M      29  4.5     None
13              Lady

In [3]:
# 3.
print('corgname: Must be a string value less than 50 characters')
print('breed: Acceptable values are Pem and Cardi')
print('gender: Acceptable values are F, M, SF, or NM')
print('weight: Must be a positive integer below 70')
print('age: Must be a positive numerical value below 20, rounded to the nearest half year')

corgname: Must be a string value less than 50 characters
breed: Acceptable values are Pem and Cardi
gender: Acceptable values are F, M, SF, or NM
weight: Must be a positive integer below 70
age: Must be a positive numerical value below 20, rounded to the nearest half year


In [4]:
# 4.
query = '''
CREATE TABLE IF NOT EXISTS ingest (
    filename VARCHAR(60) PRIMARY KEY,
    whendone TIMESTAMP NOT NULL);
'''
cursor.execute(query)
conn.commit()

In [5]:
# 5.
query = '''
ALTER TABLE corgi
ADD COLUMN IF NOT EXISTS fromfile VARCHAR(60);
'''
cursor.execute(query)
conn.commit()

df = pd.read_sql('''
SELECT
    corgname,
    fromfile
FROM corgi''', conn)
print(df)

                     corgname fromfile
0               Snappy Ginger     None
1               Golden Graham     None
2                      Kraken     None
3                 Snarla June     None
4             Smiley Shortdog     None
5                     Stanley     None
6            Holden Corgfield     None
7       Penny the Shedmonster     None
8   Baroness von Fluffyshanks     None
9                  Prometheus     None
10             Foxy Stumptail     None
11                  Drederick     None
12          Lady Wigglebottom     None
13                   Jedediah     None
14               Miss Juniper     None
15        Mary Queen of Corgs     None
16                    Oatmeal     None
17                 Cowboy Dan     None
18                Gareth Bale     None
19              Winston Smith     None
20                      Bucco     None
21                   Fruitcup     None
22                  Sweet Dee     None
23                  Stonewall     None


In [10]:
# 6.
csvname = 'newcorgis-20220618.csv'
cursor.execute(f'''
SELECT
    COUNT(*)
FROM ingest
WHERE filename = '{csvname}';
''')
y = cursor.fetchone()
if y[0] > 0:
    print(f'File {csvname} has already been processed')
else:
    print(f'File {csvname} has been accepted for processing.')

    df2 = pd.read_csv(csvname)
    df2['Gender'].replace(to_replace='Fem', value='F', inplace=True)
    df2['Breed'].replace(to_replace=['PWC', 'CWC'], value=['Pem', 'Cardi'], inplace=True)
    df2['Weight'] = round(df2['Weight'])

    for x in df2.index:
        cursor.execute(f'''
            INSERT INTO corgi (corgname, breed, gender, weight, age, fromfile)
            VALUES (
            '{df2.loc[x]['Name']}',
            '{df2.loc[x]['Breed']}',
            '{df2.loc[x]['Gender']}',
            {df2.loc[x]['Weight']},
            {df2.loc[x]['Age']},
            '{csvname}');
        ''')
        conn.commit()
        print(f"Record created for {df2.loc[x]['Name']}.")
        
    cursor.execute(f'''
        INSERT INTO ingest (filename, whendone)
        VALUES ('{csvname}', CURRENT_TIMESTAMP);
    ''')
    conn.commit()

    print(f'Ingestion complete for {csvname}')

    df = pd.read_sql(f"SELECT * FROM ingest WHERE filename = '{csvname}';", conn, index_col='filename')
    print(df)



File newcorgis-20220618.csv has already been processed


In [11]:
# 7.
dfv = pd.read_sql(f'''
    SELECT
        *
    FROM corgi
    WHERE fromfile = '{csvname}'
''', conn, index_col='corgid')
print(dfv)

               corgname  breed gender  weight  age                fromfile
corgid                                                                    
64             Francine    Pem      F      24  3.5  newcorgis-20220618.csv
65             Prudence    Pem     SF      29  4.0  newcorgis-20220618.csv
66            Fort Knox  Cardi     NM      34  3.3  newcorgis-20220618.csv
67              Gruyere    Pem     SF      22  1.5  newcorgis-20220618.csv
68         Sir Barxalot    Pem      M      26  2.0  newcorgis-20220618.csv
69          Tuning Fork  Cardi     NM      29  1.8  newcorgis-20220618.csv
70              Wiffles    Pem     SF      23  5.0  newcorgis-20220618.csv
71               Wumpus    Pem      F      26  1.2  newcorgis-20220618.csv
72      Qween Kanonball    Pem      F      29  3.5  newcorgis-20220618.csv
